### Gang Green All Time Stats
Chris McAllister

#### This notebook gathers all the Oakland Hockey Stats for all 4 GG teams, for every season that which the website has data.

#### Oultine:

##### 1) Import libraries
##### 2) Establish mapping of GG Team ID's in URL to Team Names (Gang Green 1, 2, etc.)
##### 3) Read in a CSV that converts SeasonIDs to the Season Name
##### 4) Establish an empty dataset of only the column header. We later append our data to this empty dataframe.
##### 5) For each GG team, loop over all the season that they have data on going back to 2007.
##### 6) Light data manipulation. Removing columns, create Points per Game metric, etc.
##### 7) Join dataset back to Team ID and Season ID tables so we can know what year / team name the IDs correspond to. 

In [2]:
# 1) Import libraries. All we need is Pandas
import pandas as pd

# Don't truncate dataframes (ie show every column)
pd.set_option('display.max_columns', None)


# Ignore any warning messages. 
import warnings
warnings.filterwarnings('ignore')

In [13]:
# 2) Create simple mapping table that converts the Team ID to a Team Name.
gg_team_ids = [1843, 692, 4622, 4818]
gg_team_names = ['Gang Green 1', 'Gang Green 2', 'Gang Green 3', 'Gang Green 4']

team_dim = pd.DataFrame(zip(gg_team_ids, gg_team_names), columns = ['TeamID', 'Team Name'])

team_dim

,TeamID,Team Name
0,1843,Gang Green 1
1,692,Gang Green 2
2,4622,Gang Green 3
3,4818,Gang Green 4


In [6]:
# 3) Create Mapping table for Season IDs too
season_dim = pd.read_csv('OaklandHockeySeasonDim.csv')

season_dim.head(60)

,SeasonID,Season,Year,SeasonName
0,11,Winter,2007.0,Winter 2007
1,12,Spring,2008.0,Spring 2008
2,13,Winter,2008.0,Winter 2008
3,14,Spring,2009.0,Spring 2009
4,15,Winter,2009.0,Winter 2009
5,16,Spring,2010.0,Spring 2010
6,17,Winter,2010.0,Winter 2010
7,18,NaN,NaN,
8,19,Spring,2011.0,Spring 2011
9,20,Winter,2011.0,Winter 2011


In [4]:
season_dim.shape

(48, 4)

In [5]:
# 4) Establish empty dataframe to get the column names. Next we'll append each team-season combination to this df
team_id = 692
url = 'https://stats.sharksice.timetoscore.com/display-schedule?team='+str(team_id)+'&season=' + str(50) + '&league=27&stat_class=1'

# Establish base data frame so we can later append onto it
df = pd.read_html(url)
df[1].columns = df[1].columns.droplevel()

df_main = df[1].iloc[0:0]



# 5) For each GG team, loop over every season to grab their stats and append it to our big dataframe we made above.
# This is written very inefficiently and will be pretty slow. 
for team_id in gg_team_ids:

    for season_id in season_dim['SeasonID']:
        url = 'https://stats.sharksice.timetoscore.com/display-schedule?team='+str(team_id)+'&season=' + str(season_id) + '&league=27&stat_class=1'

        # There are gaps in season IDs (for example there's no season #34). 
        # This would cause an error when reading the URL so we need to handle that with the try / except code block below. 
        try:
            df = pd.read_html(url)
            season_dim = pd.read_csv('OaklandHockeySeasonDim.csv')
            # Remove first layer of column (that just say says 'Game Results')
            #df[0].columns = df[0].columns.droplevel()
            #df[0].head()

            df[1].columns = df[1].columns.droplevel()
            df[1]['SeasonID'] = int(season_id)
            df[1]['TeamID'] = int(team_id)
            

            df_main = pd.concat([df_main, df[1]])
            #df_main['SeasonID'] = season_id

        except:
            pass
        
df_main

,Name,#,GP,Goals,Ass.,PPG,PPA,SHG,SHA,GWG,GWA,PSG,ENG,UAG,IG,IA,TG,TA,FG,SOG,SOA,Shots,PIMs,+/-,Hat,Pts,SeasonID,TeamID
0,michael colhoun,19.0,16,9,5,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,23,0,1,14,17.0,1843.0
1,Sergey Mineyev,77.0,15,4,9,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,4,0,0,13,17.0,1843.0
2,Yuri Pivovarov,8.0,15,6,7,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,2,0,0,13,17.0,1843.0
3,Matt Suidan,26.0,8,6,3,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,10,0,0,9,17.0,1843.0
4,Yury Kolomensky,73.0,16,2,6,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,16,0,0,8,17.0,1843.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,Tess Nogles,20,2,0,0,0,0,0,0,0,0,0,0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,60.0,4818.0
11,Brandon Paluzzi,71,2,0,0,0,0,0,0,0,0,0,0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,2,0,0,0,60.0,4818.0
12,Robert Price,21,2,0,0,0,0,0,0,0,0,0,0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,60.0,4818.0
13,David Raber,34,2,0,0,0,0,0,0,0,0,0,0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,60.0,4818.0


In [15]:
# 6) Data manipulation. Removing certain columns. Tweaking data types. Joing to our dimension tables. 

# Cast as integers so the join below works (otherwise it won't recognize 5.0 as 5, etc.)
df_main['SeasonID'] = df_main['SeasonID'].astype(int) 
df_main['TeamID'] = df_main['TeamID'].astype(int) 

# Convert season IDs (#40) to Season Name (Fall 2017)
df_final = pd.merge(left = df_main, right = season_dim, how = 'left', left_on = 'SeasonID', right_on = 'SeasonID')

# Only select necessarry columns
col = ['Name', '#', 'GP', 'Goals', 'Ass.', 'PPG', 'PPA', 'SHG', 'SHA', 'GWG',
       'GWA', 'PSG', 'ENG',  'PIMs', 'Hat', 'Pts', 'SeasonID', 'Season', 'Year', 'TeamID',
       'SeasonName']

df_final = df_final[col]

# Create a GPG and Pts per game metric. 
df_final['GPG'] = df_final['Goals'] / df_final['GP']
df_final['Pts_PG'] = df_final['Pts'] / df_final['GP']

df_final['SeasonID'] = df_final['SeasonID'].astype(int) 
df_final['TeamID'] = df_final['TeamID'].astype(int)

In [10]:
# Get team name from Team ID (GG 1, 3, etc.)
df_final_2 = pd.merge(left = df_final, right = team_dim, how = 'left', left_on = 'TeamID', right_on = 'TeamID')

#### 2) Gang Green Leaders in Stats

In [28]:
current_season = df_final_2[df_final_2['Season'] == 'Current']
current_season.sort_values(by = 'Pts', ascending = False).head(5)

,Name,#,GP,Goals,Ass.,PPG,PPA,SHG,SHA,GWG,GWA,PSG,ENG,PIMs,Hat,Pts,SeasonID,Season,Year,TeamID,SeasonName,GPG,Pts_PG,Team Name
1209,CHRISTOPHER MCALLISTER,19,4,5,1,0,0,0,0,1,0,0,0,2,1,6,0,Current,NaN,692,Current,1.250000,1.5,Gang Green 2
1211,Jason Moseley,22,3,1,2,0,1,0,0,0,1,0,0,0,0,3,0,Current,NaN,692,Current,0.333333,1.0,Gang Green 2
1210,BRENDAN BURKE,28,2,3,0,1,0,0,0,1,0,0,0,2,1,3,0,Current,NaN,692,Current,1.500000,1.5,Gang Green 2
554,Anders Manley,10.0,2,1,1,0,0,0,0,0,0,0,0,2,0,2,0,Current,NaN,1843,Current,0.500000,1.0,Gang Green 1
1362,ROBERT KERSHAW,57.0,2,1,1,1,0,0,0,0,0,0,0,0,0,2,0,Current,NaN,4622,Current,0.500000,1.0,Gang Green 3


#### 6) Milestone Leaders

In [92]:
games = [50, 100, 250, 500, 1000]
goals = [30, 50, 100, 200]
points = [50, 100, 300, 500]

player_grain = df_final_2.groupby(by = 'Name').agg({'GP': 'sum'
                                                  , 'Goals': 'sum'
                                                  , 'Pts': 'sum'}).reset_index()

In [97]:
games_played_df = []

for i in games:
    
    a = player_grain[(i -  player_grain['GP'] <=3) & (i -  player_grain['GP'] >= 0)]
    a['Milestone'] = str(i) + ' Games Played'
    a['GPs Remaining'] = i - a['GP']
    games_played_df.append(a)
    
games_played_df = pd.concat(games_played_df)

games_played_df.sort_values(by = 'GPs Remaining')

,Name,GP,Goals,Pts,Milestone,GPs Remaining
23,Anthony Fox,49,43,57,50 Games Played,1
14,Allen Lee,48,15,30,50 Games Played,2
225,Peter Jackson,48,12,24,50 Games Played,2
76,David Kane,47,3,13,50 Games Played,3
149,Joshua Miner,47,24,34,50 Games Played,3
240,Ryan Brookman,47,6,21,50 Games Played,3


In [98]:
goals_df = []

#pg_sorted = player_grain.sort_values(by = 'Pts', ascending = False).head(30)

for i in goals:
    
    a = player_grain[(i -  player_grain['Goals'] <=3) & (i -  player_grain['Goals'] >= 0)]
    a['Milestone'] = str(i) + ' Goals'
    a['Goals Remaining'] = i - a['Goals']
    goals_df.append(a)
    
goals_df = pd.concat(goals_df)

goals_df.sort_values(by = 'Goals Remaining')

,Name,GP,Goals,Pts,Milestone,Goals Remaining
6,Aaron Thompson,82,30,39,30 Goals,0
199,Michael Meade,41,30,52,30 Goals,0
28,BRIAN FLANIGAN JR,53,29,52,30 Goals,1
40,Bryan Ashley,25,29,45,30 Goals,1
69,Daniel Rudkevitch,31,29,45,30 Goals,1
194,Michael Dunn,52,49,65,50 Goals,1
131,Jeffrey Martin,56,28,47,30 Goals,2
232,Robert Jensen,195,28,61,30 Goals,2
5,Aaron SCHRAGE,54,48,79,50 Goals,2
218,Paul Mullen,89,27,65,30 Goals,3


In [100]:
points_df = []

#pg_sorted = player_grain.sort_values(by = 'Pts', ascending = False).head(30)

for i in points:
    
    a = player_grain[(i -  player_grain['Pts'] <=3) & (i -  player_grain['Pts'] >= 0)]
    a['Milestone'] = str(i) + ' Points'
    a['Points Remaining'] = i - a['Pts']
    points_df.append(a)
    
points_df = pd.concat(points_df)

points_df.sort_values(by = 'Points Remaining')

,Name,GP,Goals,Pts,Milestone,Points Remaining
144,Jon Fung,63,23,49,50 Points,1
243,SRDJAN ZIZIC,52,18,49,50 Points,1
103,Graham Rolak,28,26,47,50 Points,3
131,Jeffrey Martin,56,28,47,50 Points,3


In [7]:
# Ouput results to CSV
df_final_2.to_csv('OaklandHockeyData.csv', index = False)

In [8]:
df_final_2.head()

,Name,#,GP,Goals,Ass.,PPG,PPA,SHG,SHA,GWG,GWA,PSG,ENG,PIMs,Hat,Pts,SeasonID,Season,Year,TeamID,SeasonName,GPG,Pts_PG,Team Name
0,michael colhoun,19.0,16,9,5,0,0,0,0,0,0,0,0,23,1,14,17,Winter,2010.0,1843,Winter 2010,0.562500,0.875000,Gang Green 1
1,Sergey Mineyev,77.0,15,4,9,0,0,0,0,0,0,0,0,4,0,13,17,Winter,2010.0,1843,Winter 2010,0.266667,0.866667,Gang Green 1
2,Yuri Pivovarov,8.0,15,6,7,0,0,0,0,0,0,0,0,2,0,13,17,Winter,2010.0,1843,Winter 2010,0.400000,0.866667,Gang Green 1
3,Matt Suidan,26.0,8,6,3,0,0,0,0,0,0,0,0,10,0,9,17,Winter,2010.0,1843,Winter 2010,0.750000,1.125000,Gang Green 1
4,Yury Kolomensky,73.0,16,2,6,0,0,0,0,0,0,0,0,16,0,8,17,Winter,2010.0,1843,Winter 2010,0.125000,0.500000,Gang Green 1
